<a href="https://colab.research.google.com/github/maxigaarp/Gestion-de-datos-2022/blob/main/Capitulo_6_Depuracion_en_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# En R

In [ ]:
library(tidyverse)

## Strings

In [ ]:
adist("Francisca", "Francisc")

In [ ]:
Nominanombres=c("Maximiliano", "Ruth", "Luis", "Agata", "Flor", "Gabriel", "Elena", "Alberto")

In [ ]:
adist(c("Maximilano", "Rut", "Louis", "Agatha"), Nominanombres)

In [ ]:
adist(c("Maximiliaño"), Nominanombres)

In [ ]:
which.min(adist(c("Louis"), Nominanombres))

In [ ]:
which.min(adist(c("Maximilaño", "Rut", "Louis", "Agatha"), Nominanombres))

La funcion Which.min no esta hecha para aplicarse sobre matrices, solo vectores. Acá se vuelve necesario aplicar funciones a las filas, lo que podriamos hacer con un "for" pero es lento y poco elegante para el ligero cambio que queremos pedirle a R. 

Para estos casos existen las funciones (funcionales) de la familia apply. 

In [ ]:
b <- Nominanombres[apply(adist(c("Maximilaño", "Rut", "Louis", "Agatha"), Nominanombres), 1, which.min)]



In [ ]:
b

#Missing Data

In [ ]:
install.packages("missForest")
library(missForest)

In [ ]:
library(datasets)

df <- as.data.frame(Nile)


In [ ]:
nile.mis <- prodNA(df, noNA = 0.2)


In [ ]:
nile.mis

In [ ]:
nile.mis %>% mutate(id = row_number())%>%
          ggplot()+
          geom_line(aes(x=id, y=x))

In [ ]:
nile.mis %>% drop_na() %>%
          mutate(id = row_number())%>%
          ggplot()+
          geom_line(aes(x=id, y=x))

In [ ]:
nile.mis %>% mutate_at("x", function(x) replace(x, is.na(x), mean(x, na.rm = TRUE)))%>%
            mutate(id = row_number())%>%
          ggplot()+
          geom_line(aes(x=id, y=x),linetype = "dashed")+
          geom_line(aes(x=1:100,y=nile.mis$x))


In [ ]:
install.packages("zoo")
library(zoo)


In [ ]:
nile.mis %>% mutate(x2 = na.fill(x,"extend"))%>%
            mutate(id = row_number())%>%
          ggplot()+
          geom_line(aes(x=id, y=x2),linetype = "dashed")+
          geom_line(aes(x=1:100,y=nile.mis$x))


# Uniformidad

In [ ]:
a <-(10+rnorm(200))*1.4
b <-(10+rnorm(100))

In [ ]:
hist(a, breaks=50)

In [ ]:
hist(b, breaks=50)

In [ ]:
hist(c(a,b), breaks=50)

In [ ]:
sqrt((lat1-lat2)**2+(lon1-lon2)**2)

In [ ]:
a <- scale(a)
b <- scale(b)
hist(c(a,b), breaks=25)

# En SQL

In [ ]:
system("gdown https://drive.google.com/uc?id=1q089qSqKr7Ak29lUkzKSWjm2pcb_jzX0")
unzip("/content/matriculas_y_notas_anual.zip")

In [ ]:
install.packages("RSQLite")
install.packages("data.table")

In [ ]:
library(data.table)
library(RSQLite)
library(tidyverse)

In [ ]:
system("gdown https://drive.google.com/uc?id=1bxebySwiYquw1B72xi1E_YrELXzDS4cC")


In [ ]:
system("mv /content/Rendimiento2013.csv /content/content/Datos/20210223_Rendimiento_2013.csv")

In [ ]:
conn <- dbConnect(RSQLite::SQLite(), "mineduc.db")


Supongamos que queremos ver la cantidad de años que tiene cada alumno registrado a lo largo de los 18 años de datos disponibles en MINEDUC.

Filtrando la información me quedo con una tabla de alumnos con su informacion relevante y otra con los colegios.


# Cargando bases en SQL


In [ ]:
colenames <- c("RBD","AGNO","NOM_RBD","COD_REG_RBD","NOM_COM_RBD", "COD_DEPE","RURAL_RBD")
alnames <- c("MRUN","AGNO","RBD","COD_ENSE","COD_GRADO","LET_CUR","GEN_ALU", "FEC_NAC_ALU","COD_COM_ALU", "SIT_FIN_R",                      "PROM_GRAL", "ASISTENCIA")


years<-2010:2020
for (i in 1:length(years)) {
  name=gsub("%",years[i],"/content/content/Datos/20210223_Rendimiento_%.csv")
  data=fread(name)
  names(data) <- toupper(names(data))


  coles <- data %>% 
    select(colenames)%>% 
    distinct()
  
  alus <- data %>%
        select(c("MRUN","AGNO","RBD","COD_ENSE","COD_GRADO","LET_CUR",
                    "GEN_ALU", "FEC_NAC_ALU","COD_COM_ALU", if (years[i]!=2014) "SIT_FIN_R" else "SIT_FINAL_R",
                      "PROM_GRAL", "ASISTENCIA")) %>% 
        distinct()
        
  names(alus)=alnames

  apnd=if (i==1) FALSE else TRUE

  dbWriteTable(conn , name = "colegios", 
                  value = coles,
                  row.names = FALSE, header = !apnd, sep=',',append=apnd,
                  colClasses='character')

  dbWriteTable(conn , name = "alumnos", 
                  value = alus,
                  row.names = FALSE, header = !apnd, sep=',',append=apnd,
                  colClasses='character')
}

En particular podemos hacer consultas complicadas de manera rapida y sin preocuparnos por si el computador podrá darnos una respuesta antes de acabarse la memoria RAM.





In [ ]:
conn

Queremos ver los cambios de nombre para por ejemplo consolidar una base de datos de colegios.


```
select
tabla1.atributo1 as A1,
tabla2.atributo2 as A2,
...,
tabla1.atributon as AN,
AVG(tablan.atributon2) as avgatributon

From tabla1, tabla2,...,tablan

where tabla1.atributo1=tabla3.atributo1      

Group by tabla1.atributo1 

Having AVG(tablan.atributon2)>3

ORDER BY A1 DESC

Limit 1000

```



In [ ]:
dbExecute(conn,"CREATE TABLE COLESCAMBIO AS
select 
RBD,
AGNO,
NOM_RBD,
LAST_VALUE(NOM_RBD) OVER(PARTITION BY RBD) AS LNAME
from colegios;")

In [ ]:
dbListTables(conn)

In [ ]:
dbGetQuery(conn, "select * from COLESCAMBIO limit 20")

In [ ]:
system("wget https://sqlite.org/2016/sqlite-src-3110100.zip")
unzip("sqlite-src-3110100.zip")
system("gcc -shared -fPIC -Wall -Isqlite-src-3110100 sqlite-src-3110100/ext/misc/spellfix.c -o spellfix.so")
dbExecute(conn,"select load_extension('./spellfix')")

In [ ]:
dbGetQuery(conn,"
select 
RBD,
AGNO,
NOM_RBD,
LNAME,
EDITDIST3(NOM_RBD, LNAME) AS EDIT
FROM COLESCAMBIO")

In [ ]:
dbGetQuery(conn,"
select 
RBD,
NOM_RBD,
LNAME,
EDITDIST3(NOM_RBD, LNAME) AS EDIT
FROM COLESCAMBIO
GROUP BY RBD
HAVING EDIT=MAX(EDIT)
ORDER BY EDIT")

# Consolidando una base de datos

Supongamos que estamos interesados en ocupar nuevamente los datos de los alumnos, en particular por ahora: 
 * Asistencia
 * Promedio

Para asegurar que los datos tengan buena calidad debemos testearlos:
 * Cantidad de nulos (Completitud)
 * Revisar como se guardan los datos, ver que se cumplan estas relaciones (Validez)
 * Eliminar duplicados, datos con inconsistencias entre tablas (Consistencia)
 * Datos en un solo formato con escala adecuada para el problema (Uniformidad)

In [ ]:
dbGetQuery(conn,"select *
from alumnos 
limit 10")

#Nulos y completitud

In [ ]:
dbGetQuery(conn,"select 
  sum(case when MRUN is null then 1 else 0 end) MRUN,
  sum(case when AGNO is null then 1 else 0 end) AGNO,
  sum(case when RBD is null then 1 else 0 end) RBD,
  sum(case when COD_ENSE is null then 1 else 0 end) COD_ENSE,
  sum(case when LET_CUR is null then 1 else 0 end) LET_CUR,
  sum(case when GEN_ALU is null then 1 else 0 end) GEN_ALU,
  sum(case when FEC_NAC_ALU is null then 1 else 0 end) FEC_NAC_ALU,
  sum(case when COD_COM_ALU is null then 1 else 0 end) COD_COM_ALU,
  sum(case when SIT_FIN_R is null then 1 else 0 end) SIT_FIN_R,
  sum(case when PROM_GRAL is null then 1 else 0 end) PROM_GRAL,
  sum(case when ASISTENCIA is null then 1 else 0 end) ASISTENCIA
from alumnos")

## Definiendo bien los nulos

### Situacion final



```
UPDATE <NOMBRETABLA>
SET ATRIBUTO=#VALUE#
WHERE <<CONDICION>>
```



In [ ]:
dbGetQuery(conn, "
select *
from alumnos 
where SIT_FIN_R='' ")

In [ ]:
dbExecute(conn,"UPDATE alumnos
SET SIT_FIN_R = NULL
WHERE SIT_FIN_R=''")

In [ ]:
dbGetQuery(conn,"select
  sum(case when SIT_FIN_R is null then 1 else 0 end) SIT_FIN_R
from alumnos")

### Genero

In [ ]:
dbExecute(conn,"UPDATE alumnos
SET GEN_ALU = NULL
WHERE GEN_ALU=0")

### Promedio y asistencia


In [ ]:
dbGetQuery(conn,"
select SIT_FIN_R,
count(),
AVG(PROM_GRAL),
MIN(PROM_GRAL),
MAX(PROM_GRAL),
AVG(ASISTENCIA),
MIN(ASISTENCIA),
MAX(ASISTENCIA)
FROM alumnos
group by SIT_FIN_R")

Primero pequeña capsula de arreglar problemas de validez, el promedio esta como character, esto lo arreglamos con el siguiente comando.

In [ ]:
dbExecute(conn, " UPDATE alumnos
SET PROM_GRAL = CAST(replace(PROM_GRAL, ',', '.') AS NUMERIC);")

In [ ]:
dbExecute(conn,"UPDATE alumnos
SET PROM_GRAL = NULL, ASISTENCIA = NULL
WHERE SIT_FIN_R='Y' or SIT_FIN_R='T'")

Luego


In [ ]:
dbGetQuery(conn,"
select SIT_FIN_R,
count(),
AVG(PROM_GRAL),
MIN(PROM_GRAL),
MAX(PROM_GRAL),
AVG(ASISTENCIA),
MIN(ASISTENCIA),
MAX(ASISTENCIA)
FROM alumnos
group by SIT_FIN_R")

Notemos que hay alumnos aprobados con nota 0, consideraré eso como informacion faltante por lo que lo asignare a nulo.


In [ ]:
dbExecute(conn,"UPDATE alumnos
SET PROM_GRAL = NULL
WHERE PROM_GRAL=0")

In [ ]:
dbGetQuery(conn, "
select *
from alumnos 
where PROM_GRAL!=0 and ASISTENCIA=0
limit 10")

In [ ]:
dbGetQuery(conn,"
select SIT_FIN_R,
count(),
AVG(PROM_GRAL),
MIN(PROM_GRAL),
MAX(PROM_GRAL),
AVG(ASISTENCIA),
MIN(ASISTENCIA),
MAX(ASISTENCIA)
FROM alumnos
group by SIT_FIN_R")

In [ ]:
dbExecute(conn,"UPDATE alumnos
SET COD_COM_ALU	 = NULL
WHERE COD_COM_ALU	=0")

##¿Que eliminamos?

In [ ]:
dbExecute(conn,"
DELETE FROM alumnos 
WHERE MRUN IS NULL")

In [ ]:
dbExecute(conn,"
DELETE FROM alumnos 
WHERE SIT_FIN_R IS NULL")

### ¿Que pasa con la fecha de nacimiento?

In [ ]:
dbGetQuery(conn, "
select *
from alumnos 
where FEC_NAC_ALU is null limit 5")

In [ ]:
dbGetQuery(conn, "
select *
from alumnos 
where MRUN=2849761
limit 10")

Solo falta el registro pero el valor existe y puede extraerse

# Consistencia

In [ ]:
dbGetQuery(conn,"
  select MRUN,
  FEC_NAC_ALU,
  count() as N
  from alumnos
  group by MRUN,FEC_NAC_ALU
  order by N DESC
")

In [ ]:
dbGetQuery(conn,"
select *
FROM alumnos 
where MRUN=9014208
")

Parece todo bien pero no es asi

In [ ]:
dbGetQuery(conn,"
select
MRUN,
FEC_NAC_ALU
from (
  select MRUN,
  FEC_NAC_ALU,
  count() as N
  from alumnos
  group by MRUN,FEC_NAC_ALU
)
GROUP BY MRUN
having N=MAX(N) and 
  sum(case when FEC_NAC_ALU is null then 1 else 0 end) >0
")

In [ ]:
dbGetQuery(conn,"
update alumnos
set FEC_NAC_ALU= fchs
from (
select
MRUN as mrns,
FEC_NAC_ALU as fchs
from (
  select MRUN,
  FEC_NAC_ALU,
  count() as N
  from alumnos
  group by MRUN,FEC_NAC_ALU
)
GROUP BY MRUN
having N=MAX(N) and 
  sum(case when FEC_NAC_ALU is null then 1 else 0 end) >0)
where MRUN=mrns

")

In [ ]:
dbGetQuery(conn, "
select *
from alumnos 
where MRUN=2849761")

In [ ]:
dbExecute(conn, "
UPDATE alumnos
SET FEC_NAC_ALU=SUBSTRING(FEC_NAC_ALU,1,6)")

# Validez

Lo mas importante es verificar la unicidad de lo que creamos que sea la llave

In [ ]:
a<-dbGetQuery(conn,"
select 
MRUN,
AGNO, 
count() as n
from alumnos
where SIT_FIN_R='P'
group by MRUN, AGNO
having n>1 
order by n
")
a

In [ ]:
dbGetQuery(conn,"
select *
from alumnos
where MRUN=951
limit 100")

In [ ]:
a %>% filter(n==3)

In [ ]:
dbGetQuery(conn,"select * from alumnos where MRUN=3138123")

In [ ]:
dbGetQuery(conn,"
select MRUN,
AGNO, 
COD_ENSE, 
COD_GRADO,
COUNT() AS N
from alumnos
WHERE SIT_FIN_R='P'
group by MRUN,COD_ENSE, COD_GRADO
ORDER BY N")

In [ ]:
dbGetQuery(conn,"
select *
from alumnos
WHERE MRUN=9098462")

In [ ]:
dbExecute(conn,"DELETE FROM alumnos 
WHERE MRUN IN (
select 
MRUN
from alumnos
where SIT_FIN_R='P'
group by MRUN, AGNO
having count()>1 )
")

OPCION 1:

In [ ]:
dbGetQuery(conn,"
select alumnos.* 
from alumnos, (select
MRUN,
FEC_NAC_ALU
from (
  select MRUN,
  FEC_NAC_ALU,
  count() as N
  from alumnos
  group by MRUN,FEC_NAC_ALU
)
GROUP BY MRUN
having N=MAX(N)) as cumples

where alumnos.MRUN=cumples.MRUN and
alumnos.FEC_NAC_ALU!=cumples.FEC_NAC_ALU
")

OPCION 2:

In [ ]:
dbExecute(conn,"DELETE FROM alumnos 
WHERE MRUN IN (
select 
MRUN
from alumnos
group by MRUN
having count(DISTINCT FEC_NAC_ALU)>1 )")

In [ ]:
dbGetQuery(conn, "
select * from alumnos ")

## Testeo de algunas relaciones

Suponemos que los alumnos para pasar de curso deben tner el promedio azul. Podemos testear esta hipotesis sobre los datos

In [ ]:
dbGetQuery(conn,"select * from alumnos where PROM_GRAL<4 and SIT_FIN_R='P'")

Por el otro lado los reprobados deberian tener notas o asistencia baja

In [ ]:
dbGetQuery(conn, "select * from alumnos where PROM_GRAL>5.5 and ASISTENCIA>60 and SIT_FIN_R='R'")

# Preguntas

Tenemos listos los datos para ser utilizados pero dependerá del uso que vamos a querer darles. Ej:



 * Las notas por nivel

In [ ]:
resp <- dbGetQuery(conn, "
SELECT 
COD_ENSE,
COD_GRADO,
AVG(PROM_GRAL) AS MPROM
FROM alumnos
Where (COD_ENSE==110 OR COD_ENSE==310) and (SIT_FIN_R=='P')
GROUP BY COD_ENSE, COD_GRADO")

In [ ]:
resp %>% arrange(COD_ENSE,COD_GRADO)%>%
        ggplot(aes(x=1:12,y=MPROM)) + 
            geom_bar(stat="identity", position="stack")


 * Promedio de un alumno durante su enseñanza (completa)

In [ ]:
resp <- dbGetQuery(conn, "
SELECT 
MRUN,
COUNT() AS N,
AVG(ASISTENCIA) AS MASIS, 
AVG(PROM_GRAL) AS MPROM
FROM alumnos
Where (COD_ENSE==110 OR COD_ENSE==310) and (SIT_FIN_R=='P')
GROUP BY MRUN
HAVING N=12"
)

In [ ]:
ggplot(resp, aes(x=`MASIS`, y=`MPROM`)) +  geom_point(size=2, shape=23)

 * Comunas con mejor promedio

In [ ]:
resp <- dbGetQuery(conn, "
SELECT 
COD_COM_ALU,
COUNT() AS N,
AVG(ASISTENCIA) AS MASIS, 
AVG(PROM_GRAL) AS MPROM
FROM alumnos
Where (COD_ENSE==110 OR COD_ENSE==310) and (SIT_FIN_R=='P')
GROUP BY COD_COM_ALU
")

In [ ]:
arrange(resp, MPROM)

* 

# Muestreo

In [ ]:
dbGetQuery(conn, "
SELECT *
FROM alumnos
LIMIT 1000
")

Muestreo aleatorio simple

In [ ]:
dbGetQuery(conn, "
SELECT *
FROM alumnos
ORDER BY RANDOM()
LIMIT 1000
")

Muestreo Aleatorio Estratificado

In [ ]:
dbGetQuery(conn, "
  SELECT DISTINCT MRUN
  FROM alumnos
  ORDER BY RANDOM()
  LIMIT 1000
")

In [ ]:
dbGetQuery(conn, "
SELECT *
FROM alumnos
WHERE MRUN IN (
  SELECT DISTINCT MRUN
  FROM alumnos
  ORDER BY RANDOM()
  LIMIT 1000)
")

In [ ]:
sample.int(3,1)-1


In [ ]:
a=3002
b=sample.int(a,1)-1

dbGetQuery(conn, "
select * 
FROM alumnos
WHERE MRUN %? =?
limit 10
",  params = c(a,b))

## Estratificado

In [ ]:
dbGetQuery(conn, "
select COD_COM_ALU, count() as n
FROM alumnos
group by COD_COM_ALU
")

In [ ]:
sample <- 10000

a <- dbGetQuery(conn, "
select COD_COM_ALU, ?*count(*)/ CAST( SUM(count(*)) over () as float) as PERC
FROM alumnos
group by COD_COM_ALU
", params=c(sample))

In [ ]:
sample <- 10000

for (row in 1:2) {

  sub <- dbGetQuery(conn, "
  select * 
  from alumnos
  where COD_COM_ALU=?
  order by RANDOM()
  limit ?
  ", params=c(a[row, "COD_COM_ALU"],ceiling(a[row, "PERC"])))

   out=if(row==1) sub else rbind(out,sub)

}

In [ ]:
out

In [ ]:
rbind(sub,sub)

# Clasificacion

Ir a https://www.cr2.cl/datos-de-precipitacion/?cp_Precipitacion=2

Descargar datos del 2019

In [ ]:
library(tidyverse)
library(data.table)

In [ ]:
unzip("/content/cr2_prDaily_2018.zip")
unzip("/content/cr2_tasDaily_2020_ghcn.zip")

In [ ]:
pp <- read.csv("/content/cr2_prDaily_2018/cr2_prDaily_2018.txt", na = "-9999", header =F)
tm <- read.csv("/content/cr2_tasDaily_2020_ghcn/cr2_tasDaily_2020_ghcn.txt", na = "-9999", header =F)


In [ ]:
head(pp)

In [ ]:
tm <- setNames(as.data.frame(t(tm[,-1])),as.character(tm[,1]))
pp <- setNames(as.data.frame(t(pp[,-1])),as.character(pp[,1]))


In [ ]:
ppp <- pp %>% select( c("codigo_estacion","nombre", "latitud","longitud") | "2000-01-01":"2017-12-31")%>%
        pivot_longer(cols = "2000-01-01":"2017-12-31",
        values_to = "Precipitacion",
        names_to = c("Año", "Mes", "Dia"),
        names_pattern = "(....)-(..)-(..)")

ppp

In [ ]:
tmp <- tm %>% select( c("codigo_estacion") | "2000-01-01":"2017-12-31")%>%
        pivot_longer(cols = "2000-01-01":"2017-12-31",
        values_to = "Temperatura",
        names_to = c("Año", "Mes", "Dia"),
        names_pattern = "(....)-(..)-(..)")

tmp

In [ ]:
X_full <- ppp %>%
        inner_join(tmp, by=c("codigo_estacion","Año",	"Mes","Dia"))%>%
        mutate(Precipitacion=as.double(Precipitacion),
               Temperatura=as.double(Temperatura),        
               BPrecipitacion=ifelse(Precipitacion>0,1, 0)) %>%        
        group_by(codigo_estacion)%>%
        mutate(YPrecipitacion=shift(Precipitacion,1),
               YTemperatura=shift(Temperatura,1)) %>%
        ungroup()%>%
        group_by(codigo_estacion,Mes,Dia )%>%
        mutate(MPrecipitacion=mean(Precipitacion,na.rm=TRUE),
               MTemperatura=mean(Temperatura,na.rm=TRUE)) %>%        
        ungroup()
              


In [ ]:
df <-apply(X = is.na(X_full), MARGIN = 2, FUN = mean)
print(df*100)


## KNN

In [ ]:
install.packages("caTools")

In [ ]:
library(class)
library(caTools)

In [ ]:
X<- X_full%>%
        filter(Año==2017& Mes==12 & Dia==14)%>%
        select("latitud", "longitud","BPrecipitacion")%>%
        drop_na()%>%
        select("latitud", "longitud")


In [ ]:
Y<- X_full%>%
        filter(Año==2017& Mes==12 & Dia==14)%>%
        select("latitud", "longitud","BPrecipitacion")%>%
        drop_na()%>%
        select("BPrecipitacion")


In [ ]:
set.seed(101) 
sample = sample.split(X$longitud, SplitRatio = .75)

X_train = subset(X, sample == TRUE)
X_test  = subset(X, sample == FALSE)
Y_train = subset(Y, sample == TRUE)
Y_test  = subset(Y, sample == FALSE)

In [ ]:
knn.prd=knn(X_train,X_test,Y_train[["BPrecipitacion"]],k=5,prob=TRUE)
table(knn.prd,Y_test[["BPrecipitacion"]])

Acá calculamos los indicadores.

# Regresion Logistica

In [ ]:
head(X_full)

In [ ]:
datax <- X_full%>% 
        select(c("BPrecipitacion", "Temperatura", "YPrecipitacion", "YTemperatura", "MPrecipitacion", "MTemperatura"))%>%
        drop_na()

set.seed(101) 
sample = sample.split(datax$BPrecipitacion, SplitRatio = .75)

train = subset(datax, sample == TRUE)
test  = subset(datax, sample == FALSE)

In [ ]:
m1 <- glm(BPrecipitacion ~ YPrecipitacion+YTemperatura+MPrecipitacion , family = binomial,data=train)
summary(m1)

In [ ]:
pred<- predict.glm(m1,newdata = test, type="response")
result1<- table(test$BPrecipitacion, floor(pred+0.5))
result1